# Final exam: 

**Instructions:** the exam is composed of three exercises including theoretical and implementation parts. Every exercise can be treated independently.
- This document is available from the 18th March at 8 AM and must be uploaded back **BEFORE the 26th at 8 PM** (take your precautions to upload it in time).   
- Answering in markdown in the notebook is encouraged, but if you have difficulties with it, you are allowed to answer the theoretical part on a separate document in pdf, png, or jpg format.
- In both cases, the clarity (clean writing) and the organization of the work (numbering the questions appropriately) are taken into account during the correction. 
- You are allowed to use any document (script, slides, other internet sources...).
- Before uploading your notebook, **clean it (Kernel/restart and clear output)** to make it  lighter. And **verify that your code is running cell after cell.**

In [ ]:
#library and functions used in the notebook
import numpy as np
import matplotlib.pyplot as plt
import numpy.linalg as lin
from scipy.linalg import cho_factor, cho_solve

## Exercise 1: Convergence of tridiagonal Jacobi and Gauss-Seidel methods

We aim to compare numerically the convergence rates of Jacobi and Gauss-Seidel algorithms in the case of tridiagonal matrices. We focus on one example given by the matrix of the Laplacian 

$$L = \left(\begin{array}{cccccc} 
2  & -1 & 0  & \dots & \dots & 0 \\
-1 & 2  & -1 & 0 &  & \vdots \\
0  & -1 & 2  & -1 & \ddots & \vdots \\
\vdots & \ddots & \ddots & \ddots & \ddots & 0  \\ 
\vdots &  & \ddots & \ddots & \ddots& -1  \\ 
0 & \dots & \dots & 0 & -1 & 2
\end{array}\right) \in \mathbb{R}^{N\times N}.$$

1) a) Implement Jacobi algorithm to solve systems of the form $L V = b$. 
The arguments of the function are given. There are two stopping criteria, one on the maximal number of iterations and one on the error $\|AV^n-b\|$.

*Hint: you may exploit the considered tridiagonal structure of the matrix $L$.*

In [ ]:
def Jacobi_Laplace(N, b, V0, it_max = 10**4, TOL = 10**(-10)):
    """
    Compute the iterations of Jacobi algorithm for the Laplacian matrix
    ----------   
    parameters:
    N      : size of the matrix NxN
    b      : RHS of the problem LV = b where L is the Laplacian matrix
    V0     : initial vector in the method
    it_max : maximum number of iterations
    TOL    : use || L V^k - b || < TOL as a stopping criteria 
    
    returns:
    V       : the solution at the end of the iterations
    tab_err : array of the || L V^k - b ||^2 at every iteration 
    """
    
    #initialization 
    V_new      = np.copy(V0) 
    V          = np.copy(V_new) 
    tab_err    = np.zeros(it_max + 1) 

    #convergence loop 
    k          = 0 
    LV         = matmulLV(N, V_new) 
    err        = lin.norm(LV - b) 
    tab_err[0] = err 
    while (k < it_max) and (err > TOL): 
        k += 1 
        V = np.copy(V_new) 
        
        V_new[0]   = (b[0] + V[1]) / 2 
        V_new[N-1] = (b[N-1] + V[N-2]) / 2 
        for i in range(1, N-1): 
            V_new[i] = (b[i] + V[i-1] + V[i+1]) / 2 
        
        LV         = matmulLV(N, V_new) 
        err        = lin.norm(LV - b) 
        tab_err[k] = err 
    
    return V, tab_err[:k+1] 


def matmulLV(N, V_new): 
    """
    Compute the matrix multiplication (L, V_new) in order to compute the error at each iteration. 
    ---------- 
    Parameters: 
    N     : size of the matrix. 
    V_new : the V_new matrix at each iteration. 
    
    returns: 
    LV : the matrix multiplication L, V_new. 
    """
    LV = np.copy(V_new) 
    
    LV[0]   = 2*V_new[0] - V_new[1] 
    LV[N-1] = - V_new[N-2] + 2*V_new[N-1] 
    for i in range(1, N-1): 
        LV[i] = - V_new[i-1] + 2*V_new[i] - V_new[i+1] 
    
    return LV 

b) Test it with the given data, $N= 20$, $b= (1,0,\dots,0)$ and plot the error obtained with this method as a function of the iteration $n$ in logscale.

In [ ]:
#parameters
N = 20 
b = np.zeros(N); b[0] = 1.

#solving the system
sol_J, err_J = Jacobi_Laplace(N, b, b, it_max = 10**4, TOL = 10**(-10))

#plot the errors
plt.figure(1)
plt.yscale("log")
plt.xlabel("Iteration")
plt.ylabel("Error")
plt.plot(err_J)
plt.show()

2) a) Implement Gauss-Seidel algorithm to solve the system $L V = b$ where $b = (1, 0, \dots, 0) \in\mathbb{R}^N$. 
The arguments of the function are given.


*Hint: you may exploit again the considered tridiagonal structure of the matrix.*

In [ ]:
def GS_Laplace(N, b, V0, it_max = 10**4, TOL = 10**(-10)):
    """
    Compute the iterations of Gauss-Seidel algorithm for the Laplacian matrix
    ----------   
    parameters:
    N      : size of the matrix NxN
    b      : RHS of the problem L V = b
    V0     : initial vector in the method
    it_max : maximum number of iterations
    TOL    : use || L V^k - b || < TOL as a stopping criteria 
    
    returns:
    V       : the solution at the end of the iterations
    tab_err : array of the || L V^k - b ||^2 at every iteration 
    """
    
    #initialization
    V_new   = np.copy(V0)
    V       = np.copy(V_new)
    tab_err = np.zeros(it_max+1)

    #convergence loop
    k          = 0 
    LV         = matmulLV(N, V_new) 
    err        = lin.norm(LV - b) 
    tab_err[0] = err 
    while (k < it_max) and (err > TOL): 
        k += 1 
        V = np.copy(V_new) 
        
        V_new[0] = (b[0] + V[1]) / 2 
        for i in range(1, N-1): 
            V_new[i] = (b[i] + V_new[i-1] + V[i+1]) / 2 
        V_new[N-1] = (b[N-1] + V_new[N-2]) / 2 
        
        LV         = matmulLV(N, V_new) 
        err        = lin.norm(LV - b) 
        tab_err[k] = err 
    
    return V, tab_err[:k+1] 

b) Test it with the given data, $N= 20$, $b= (1,0,\dots,0)$. Compare the behaviour (convergence rate, number of iterations, ...) of Jacobi and Gauss-Seidel algorithms on this test case.

In [ ]:
#parameters
N = 20 
b = np.zeros(N); b[0] = 1.

#solving the system
sol_GS, err_GS = GS_Laplace(N, b, b, it_max = 10**4, TOL = 10**(-10))

#plot the errors
plt.figure(2)
plt.yscale("log")
plt.xlabel("Iteration")
plt.ylabel("Error")
plt.plot(err_J)
plt.plot(err_GS)
plt.legend(["Jacobi","Gauss-Seidel"])
plt.show()

**Answer:** 

- We see that the convergence rate of the Gauss-Seidel algorithm is faster than the convergence rate of the Jacobi algorithm, because the slope of the yellow curve is more steep than the slop of the blue curve. 

- We see that the number of iterations required of the Gauss-Seidel algorithm is less than the number of iterations required of the Jacobi algorithm, which is clearly indicated by the graph above on the x-axis. 

c) Compute numerically the spectral radii of Jacobi and Gauss-Seidel iteration matrices using the convergence rates of those methods on the last test case. Compare them for different values of N. 

(Bonus): For the considered matrix, the two spectral radii are related to each others (one is a function of the other). Find numerically this relation.   

In [ ]:
rho_J  = err_J[-1] / err_J[-2] 
print(f"rho Jacobi       : {rho_J}"   )

rho_GS  = err_GS[-1] / err_GS[-2] 
print(f"rho Gauss-Seidel : {rho_GS}"  )

**Answer:** 

For $N = 20$, the spectral radii are 
- Jacobi : 0.9888311798960567
- Gauss-Seidel : 0.9777864458910915 

For $N = 100$, the spectral radii are 
- Jacobi : 0.9995162822925521
- Gauss-Seidel : 0.9990327985678236

Therefore, we see that as $N$ increases, the spectra radius of both Jacobi and Gauss-Seidel algorithm gradually converges to 1. Moreover, the spectral radii of Jacobi is greater than the spectra radii of Gauss-Seidel, implying that for our chosen $N$, the Gauss-Seidel algorithm converges faster than the Jacobi algorithm. 

---
---

## Exercise 2: Complexity of Strassen's multiplication

**Historical context:** In the late 60's, the operations performed by computers were slower, especially, multiplications in floating point arithmetic was around three times slower than additions. To compensate that, Winograd in 1967, then Strassen in 1969 studied factorizations of matrix products in order to dicrease the number of operations that these required (especially the number of multiplications). This factorization also lead to direct methods for the computation of inverse matrices that required less operations (for large matrices $N\rightarrow \infty$) than the classical Gaussian elimination.

In this exercise, we compare at a theoretical level the number of operations (additions and multiplications together) required when using the basic formula of matrix-matrix product and when using Strassen formulae. 

### Part 1: Basic computation of the product

1) Consider two vectors $V\in\mathbb{R}^N$ and $W\in\mathbb{R}^N$. How many operations are required to compute their dot product?

**Answer:** 

Their dot product is computed as 

$$ 
V \cdot W = 
\begin{pmatrix} 
    V_1 \\ V_2 \\ \vdots \\ V_N 
\end{pmatrix} \cdot 
\begin{pmatrix} 
    W_1 \\ W_2 \\ \vdots \\ W_N 
\end{pmatrix} = 
V_1 W_1 + V_2 W_2 + \cdots + V_N W_N 
$$ 

Hence, there are in total $N$ multiplications and $N - 1$ additions. Therefore, the total operations required is $2N - 1$. 

2) Deduce the number of operations required to compute the matrix product $AB$ of two matrices of $\mathbb{R}^{N\times N}$ by computing each of its component as the dot product of two vectors.

**Answer:** 

Each component of the matrix $AB$ requires $2N - 1$ operations as the dot product of two vectors. Since the matrix $AB$ has in total $N^2$ components, we can deduce that the total operations required is 

$$ 
(2N - 1) \times N^2 = 2N^3 - N^2 
$$ 

3) Compare the number of operations required to do the product of two $N\times N$ matrices with the one of two $2N\times 2N$ matrices when $N$ is large.

**Answer:** 

The number of operations to compute the product of two $N \times N$ matrices is $M_{N} = 2N^3 - N^2$. 

The number of operations to compute the product of two $2N \times 2N$ matrices is $M_{2N} = 2 (2N)^3 - (2N)^2 = 16N^3 - 4N^2$. 

Hence, we obtain that $M_{2N} = 8 M_{N} + 4 N^2$. 

Moreover, when $N$ is large, we know that the $N^3$ term will dominate and $N^2$ term is negligible. As a result, we can approximate it as $M_{2N} \approx 8 M_{N}$. Therefore, computing the product of two $2N \times 2N$ matrices requires 8 times more operations than computing the product of two $N \times N$ matrices. 

### Part 2: Strassen's factorization

Strassen's formulae is provided but the  construction is omited here. Only the study of its complexity is focussed on. We will compute the number of operations required through this factorization and compare it with the one obtain in the previous section.

The matrices $A$ and $B$ are of size $\mathbb{R}^N$ where $N = 2^p$ is some power of 2. Decompose all matrices into block form of equal size 

$$ A = \left(\begin{array}{c|c} A_{1,1} & A_{1,2} \\ \hline A_{2,1} & A_{2,2}\end{array}\right), \quad B = \left(\begin{array}{c|c} B_{1,1} & B_{1,2} \\ \hline B_{2,1} & B_{2,2}\end{array}\right), \quad C = AB = \left(\begin{array}{c|c} C_{1,1} & C_{1,2} \\ \hline C_{2,1} & C_{2,2}\end{array}\right),$$

where $A_{i,j}$, $B_{i,j}$ and $C_{i,j}$ are submatrices of size $\frac{N}{2}\times \frac{N}{2}.$

Then Strassen provided the following 7 matrices 

\begin{align*}
    P_1 &= (A_{1,1}+A_{2,2})(B_{1,1}+B_{2,2}) \\
    P_2 &= (A_{2,1}+A_{2,2,})B_{1,1} \\
    P_3 &= A_{1,1} (B_{1,2}-B_{2,2}) \\
    P_4 &= A_{2,2}(B_{2,1}-B_{1,1}) \\
    P_5 &= (A_{1,1}+A_{1,2})B_{2,2} \\
    P_6 &= (A_{2,1}-A_{1,1})(B_{1,1}+B_{1,2}) \\
    P_7 &= (A_{1,2}-A_{2,2})(B_{2,1}-B_{2,2})
\end{align*}

such that the matrix product $C= AB$ yields 

\begin{align*}
    C_{1,1} &= P_1 + P_4 - P_5 + P_7 &= A_{1,1}B_{1,1} + A_{1,2} B_{2,1} \\
    C_{1,2} &= P_3 + P_5             &= A_{1,1}B_{1,2} + A_{1,2} B_{2,2} \\
    C_{2,1} &= P_2 + P_4             &= A_{2,1}B_{1,1} + A_{2,2} B_{2,1} \\
    C_{2,2} &= P_1 + P_3 - P_2 + P_6 &= A_{2,1}B_{1,2} + A_{2,2} B_{2,2}.
\end{align*}

We aim to compute the number of operations required to obtain $C$ through the knowledge of the $P_i$'s.

For this purpose, write $S_{N}$ the number of operations required to compute the sum of two matrices of size $N$ and $M_N$ the number of operations required to compute the multiplication of two matrices.

4) Compute $S_N$.  

**Answer:** 

To sum two matrices, we simply need to add each component of the two matrices together. Since there are in total $N^2$ components for each matrix, and each one of them requires only one addition, we can conclude that the total number of operations required is $S_N = N^2$. 

5) Assuming that $A$ and $B$ are of size ${N\times N}$, how many operations are required to compute $P_1$ (or equivalently $P_6$ or $P_7$) ? Express it as a function of $S_{N/2}$ and $M_{N/2}$

**Answer:** 

For computing $P_1$ (or equivalently $P_6$, $P_7$), we need to sum two matrices of size $N/2$ twice and multiply two matrices of size $N/2$ once. Thus, the total number of operations required is $2 S_{N/2} + M_{N/2}$. 

6) Assuming that $A$ and $B$ are of size ${N\times N}$, how many operations are required to compute $P_2$ (or equivalently $P_3$, $P_4$ or $P_5$) ? Express it as a function of $S_{N/2}$ and $M_{N/2}$

**Answer:** 

For computing $P_2$ (or equivalently $P_3$, $P_4$, $P_5$), we need to sum two matrices of size $N/2$ once and multiply two matrices of size $N/2$ once. Thus, the total number of operations required is $S_{N/2} + M_{N/2}$. 

7) Knowing $P_i$ for $i=1,...,7$, how many operations are required to obtain $C = AB$.

**Answer:** 

By definition, we know that 
- $C_{1,1}$ is the sum of 4 matrices of size $N/2$. Hence, the number of operations required is $3 S_{N/2}$. 
- $C_{1,2}$ is the sum of 2 matrices of size $N/2$. Hence, the number of operations required is $S_{N/2}$. 
- $C_{2,1}$ is the sum of 2 matrices of size $N/2$. Hence, the number of operations required is $S_{N/2}$. 
- $C_{2,2}$ is the sum of 4 matrices of size $N/2$. Hence, the number of operations required is $3 S_{N/2}$. 

Therefore, the total number of operations required to compute $C$ is $8 S_{N/2}$ given $P_i$ for $i = \{ 1, \dots, 7 \}$. 

8) Deduce $M_{N}$ as a function of $M_{N/2}$ and $N$.

**Answer:** 

The total number of operations required to compute $C$ is the sum of the number of operations required to compute $P_i$ for $i \in \{ 1, \dots, 7 \}$ and the number of operations required to compute $C$ given $P_i$. Therefore, we obtain that 

$$ 
M_{N} = 3 \left( 2 S_{N/2} + M_{N/2} \right) + 4 \left( S_{N/2} + M_{N/2} \right) + 8 S_{N/2} = 18 S_{N/2} + 7 M_{N/2} 
$$ 

Since $S_{N/2} = \left( \frac{N}{2} \right)^2 = \frac{N^2}{4}$, we can conclude that 

$$ 
M_{N} = 18 \left( \frac{N^2}{4} \right) + 7 M_{N/2} = 4.5 N^2 + 7 M_{N/2} 
$$ 

9) Compare this raise with the one found in **Part 1-3)** when $N$ is large. Does Strassen's method constitute an improvement?

**Answer:** 

Given $M_{N}$ requiring $\propto N^3$ number of operations, we can deduce that 

- For the Strassen's method, $M_{2N} = 7 M_{N} + 18 N^2 \approx 7 M_{N}$ when $N$ is large. 

- For the basic computation method, $M_{2N} = 8 M_{N} + 4 N^2 \approx 8 M_{N}$ when $N$ is large. 

Therefore, we can see that Strassen's method requires less operations and thus it constitutes an improvement. 

---
---

## Exercise 3: Incomplete Cholesky factorization

In this exercise, we aim to accelerate the convergence of an iterative method by exploiting a direct method. We illustrate this method again on a matrix closely related to Laplacian, i.e. 

$$ A_{i,j} = 2 \left(1+\frac{1}{N}\right) \delta_{i,j} - \delta_{i+1,j} - \delta_{i-1,j} - \delta_{i,1}\delta_{j,N} - \delta_{i,N}\delta_{j,1}. \qquad{} (3.1)$$

The construction of this matrix is implemented in the following function:

In [ ]:
def Construct_Matrix(N):
    #Construct the desired matrix of size NxN
    A  = 2.*(1+1/N)*np.eye(N)-np.diag(np.ones(N-1),1)-np.diag(np.ones(N-1),-1)
    A[0,-1] -= 1.
    A[-1,0] -= 1.  
    return A

print("Matrix(10): \n",Construct_Matrix(10))

### Part 1: Foreword

1) Consider the problem $AV = b$.
Prove that Jacobi algorithm converges toward $V^\infty = A^{-1}b$ for all initialization $V^0$. 

**Answer:** 

First, we prove that $A$ is diagonally strictly dominant. By definition, we know that 

$$
A = \left(\begin{array}{cccccc} 
2 + \frac{2}{N} & -1 & 0  & \dots & \dots & -1 \\
-1 & 2 + \frac{2}{N} & -1 & 0 &  & \vdots \\
0  & -1 & 2 + \frac{2}{N} & -1 & \ddots & \vdots \\
\vdots & \ddots & \ddots & \ddots & \ddots & 0  \\ 
\vdots &  & \ddots & \ddots & \ddots& -1  \\ 
-1 & \dots & \dots & 0 & -1 & 2 + \frac{2}{N} 
\end{array}\right) 
$$

Fix a row $i \in \{ 1, \dots, N \}$. Then, 
- The diagonal component in row $i$ is $A_{i,i} = 2 \left( 1 + \frac{1}{N} \right) = 2 + \frac{2}{N}$. 
- The sum of the remaining components in row $i$ is $\sum_{j \neq i} \left| A_{i,j} \right| = 2$. 

Since $\left| A_{i,i} \right| > \sum_{j \neq i} \left| A_{i,j} \right|$ for every row in the matrix $A$, we deduce that $A$ is diagonally strictly dominant. Therefore, the Jacobi algorithm always converges for all initialization $V^0$. 

2) Jacobi algorithm is implemented (see below) and tested in the next cells. Study this implementation and, based on it, compute numerically the convergence rate of this method. What would be the ideal convergence rate ? And the worst possible such that the method remains convergent ? Is the present convergence fast or slow ?

In [ ]:
def Jacobi(A, b, V0, it_max=1000, TOL=10**(-10)):
    """
    Compute the iterations of Jacobi algorithm
    ----------   
    parameters:
    A      : Matrix of size NxN of the problem A V = b
    b      : RHS of the problem A V = b
    V0     : initial vector in the method
    it_max : maximum number of iterations
    TOL    : use || A V^k - b || < TOL as a stopping criteria 
    
    returns:
    V       : the solution at the end of the iterations
    tab_err : array of the || A V^k - b ||^2 at every iteration
    """
   
    # initializations
    V_new      = np.copy(V0)
    tab_err    = np.zeros(it_max)
    tab_err[0] = lin.norm(np.matmul(A,V_new) - b)
    
    # construction of the matrices
    D = np.diag(A)
    R = np.diag(D) - A
    
    # Convergence loop
    for k in range(it_max-1):
        V = np.copy(V_new)
        
        if(tab_err[k] < TOL): break
        
        V_new = (b + np.dot(R, V)) / D
        
        tab_err[k+1] = lin.norm(np.matmul(A,V_new) - b)
        
    return V, tab_err[:k]

In [ ]:
# parameters
N  = 100
A  = Construct_Matrix(N)
b  = np.ones(N)
V0 = np.zeros(N) 

# solving the linear system
U_Jacobi, err_Jacobi = Jacobi(A, b, V0)

#plotting the error 
plt.figure(3)
plt.yscale("log")
plt.xlabel("Iteration")
plt.ylabel("Error")
plt.plot(err_Jacobi)
plt.show()

In [ ]:
# Computing the convergence rate
rho_Jacobi = err_Jacobi[-1] / err_Jacobi[-2] 
print(f"rho_Jacobi {rho_Jacobi}")

**Answer:** 

The ideal convergence rate is when the spectral radius is close to $0$. 

The worst convergence rate is when the spectral radius is close to $1$. 

Given the spectral radius, we can deduce the convergence rate, which is similar to the sequence $\left( \rho (A)^k \right)_k$. 

Since the spectral radius of the Jacobi algorithm is $0.9900990099831737$ as computed above for $N = 100$, we can conclude that it converges slowly. 

### Part 2: Acceleration

Now we aim to accelerate the convergence of Jacobi algorithm using an incomplete Cholesky factorization.

3) The principle of the incomplete Cholesky factorization is to write a symmetric positive definite matrix $A$ under the form 
$$A = L L^T - R$$
where $L$ is lower triangular (and ideally $L$ is sparse, i.e. it has many zero entries and its computation is expected to require less operations). 

Consider the matrix $A$ defined in (3.1) and the matrix $R$ given by $$R_{i,j} = \delta_{i,1}\delta_{j,N} + \delta_{i,N}\delta_{j,1}.$$
Show that $A$ possesses an incomplete Cholesky factorisation $A = LL^T - R$. 

**Answer:** 

By definition, we know that 

$$
A + R = \left(\begin{array}{cccccc} 
2 + \frac{2}{N} & -1 & 0  & \dots & \dots & 0 \\
-1 & 2 + \frac{2}{N} & -1 & 0 &  & \vdots \\
0  & -1 & 2 + \frac{2}{N} & -1 & \ddots & \vdots \\
\vdots & \ddots & \ddots & \ddots & \ddots & 0  \\ 
\vdots &  & \ddots & \ddots & \ddots& -1  \\ 
0 & \dots & \dots & 0 & -1 & 2 + \frac{2}{N} 
\end{array}\right) 
$$

Hence, the matrix $A + R$ is diagonally strictly dominant. It means that the Gershorin circle of the matrix $A + R$ contains only positive numbers, implying that all the eigenvalues of $A + R$ are positive. Thus, we know that the matrix $A + R$ is definite positive. Moreover, it is also symmetric. Therefore, it possesses an Imcomplete Cholesky factorization, meaning that there exists a lower triangular matrix $L$ such that $A + R = L L^T$. 

4) Consider the following stationary iteration method: with $A = M-R = LL^T-R$ as above, define the sequence $V^k$ by choosing $V^0\in \mathbb R^N$ and setting $MV^{k+1} = b + RV^k$. 

a) Write down the resulting iterative method obtained by choosing the matrix of the iteration method $M = L L^T$ as the one of the incomplete Cholesky decomposition. What algorithms would you use to compute $V^{k+1}$?

b) Implement this method and test it with the same parameter as in 3). 

*Hints:*
- *in order to simplify the implementation, you may use the functions cho_factor and cho_solve from scipy.linalg to compute the incomplete Cholesky factorization and to compute $M^{-1}$. Reading the documentation (https://docs.scipy.org/doc/scipy/reference/linalg.html) of those functions is recommended before using them.*
- *You may exploit the implementation of Jacobi algorithm (given above) to construct this one.*

c) Compute its convergence rate numerically and compare it with the one of Jacobi method.

**Answer:** 

a) By choosing $M = L L^T$, we obtain that $L L^T V^{k+1} = b + R V^{k}$. We will compute $V^{k+1}$ in the following way. 

First, we compute the value of $V_1 = L^T V^{k+1}$ by solving the algorithm 

$$ 
L V_1 = B \qquad \text{with} \quad B = b + R V^{k} 
$$ 

Next, we compute the value of $V^{k+1}$ by solving the algorithm 

$$ 
L^T V^{k+1} = V_1 \qquad \Longrightarrow \qquad V^{k+1} = \left( L^T \right)^{-1} V_1 
$$ 

Hence, we could use this method to obtain the values of $V^{k+1}$. 

b) Please see the code below. 

c) We compute the spectral radii for both algorithm. Since the spectral radius of the Incomplete Cholesky algorithm is smaller than the spectral radius of the Jacobi algorithm, we can conclude that the Incomplete Cholesky algorithm converges faster. 

In [ ]:
def Incomplete_Cholesky_iterative(A, b, V0, it_max=1000, TOL=10**(-10)):
    """
    Compute the iterations of the incomplete LU preconditioned Jacobi algorithm
    ----------   
    parameters: 
    A      : Matrix of size NxN of the problem A V = b
    b      : RHS of the problem A V = b
    V0     : initial vector in the method
    it_max : maximum number of iterations
    TOL    : use || A V^k - b || < TOL as a stopping criteria 
    
    returns: 
    V       : the solution at the end of the iterations
    tab_err : array of the || A V^k - b ||^2 at every iteration
    """
    
    # initializations 
    V_new      = np.copy(V0) 
    V          = np.copy(V_new) 
    tab_err    = np.zeros(it_max) 
    tab_err[0] = lin.norm(np.matmul(A, V_new) - b) 
    
    # construction of the matrices 
    N = len(b) 
    
    R         = np.zeros((N, N)) 
    R[0, N-1] = 1 
    R[N-1, 0] = 1 
    
    L = cho_factor(A + R) 
    
    # convergence loop
    for k in range(it_max-1):
        V = np.copy(V_new) 
        
        if(tab_err[k] < TOL): break 
        
        RV = np.matmul(R, V) 
        V_new = cho_solve(L, b+RV) 
        
        tab_err[k+1] = lin.norm(np.matmul(A, V_new) - b) 
    
    return V, tab_err[:k] 

In [ ]:
#parameters
N  = 100
A  = Construct_Matrix(N)
b  = np.ones(N)
V0 = np.zeros(N)

#solving the system
V_PJacobi, err_PJacobi = Incomplete_Cholesky_iterative(A, b, V0)

#plot the errors
plt.figure(4)
plt.yscale("log")
plt.xlabel("Iteration")
plt.ylabel("Error")
plt.plot(err_Jacobi)
plt.plot(err_PJacobi)
plt.legend(["Jacobi","Incomplete Cholesky iterative"])
plt.show()

In [ ]:
rho_IChoIt = err_PJacobi[-1] / err_PJacobi[-2] 
print(f"rho Jacobi = {rho_Jacobi}") 
print(f"rho IChoIt = {rho_IChoIt}") 